In [1]:
import os
os.environ['HUGGINGFACE_HUB_CACHE'] =  f"{os.environ['HOME']}/llm/cache"
import torch
print('CUDA Check - # of devices=',torch.cuda.device_count())
from transformers import LlamaTokenizer, LlamaForCausalLM, AutoConfig
model_id = f"{os.environ['LLM_CACHE_PATH']}/{os.environ['MODEL']}"      # /meta-llama/Llama-2-7b-chat-hf/"
config = AutoConfig.from_pretrained(model_id, trust_remote_code=True, use_auth_token=True)
tokenizer = LlamaTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, load_in_8bit=True, device_map="auto")
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

CUDA Check - # of devices= 1


/opt/conda/lib/python3.12/site-packages/transformers/models/auto/configuration_auto.py:1081: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    device_map="auto",
    pad_token_id=tokenizer.eos_token_id,
    max_length=2048,
    # temperature=1,
    # top_p=0.95,
    # repetition_penalty=1.15
)

local_llm = HuggingFacePipeline(pipeline=pipe)
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory#Summary

Device set to use cuda:0
/scratch/local/u6040150/4410374/ipykernel_353572/3668508690.py:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  local_llm = HuggingFacePipeline(pipeline=pipe)


In [4]:
memory = ConversationSummaryBufferMemory(llm=local_llm, max_token_limit=512)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
conversation = ConversationChain(
    llm=local_llm,
    memory = memory,
    verbose=False
)

conversation.prompt.template='''[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

CONTEXT:

{history}

Question: {input}[/INST]'''

class ChatBot:
    exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")

    #Method to start the conversation
    def start_chat(self):
        user_response = input("Chat here!\n")
        while user_response == '':
            user_response = input("Chat here!\n")
        self.chat(user_response)

    #Method to handle the conversation
    def chat(self, reply):
        while not self.make_exit(reply):
            input_ = reply
            reply = input(f"{conversation.predict(input = input_)}\n")

    #Method to check for exit commands
    def make_exit(self, reply):
        for exit_command in self.exit_commands:
            if exit_command in reply.lower():
                memory.clear()
                torch.cuda.empty_cache()
                print("Ok, have a great day!")
                return True
        return False

/scratch/local/u6040150/4410374/ipykernel_353572/1944454211.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(llm=local_llm, max_token_limit=512)
/scratch/local/u6040150/4410374/ipykernel_353572/1944454211.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [ ]:
chatbot = ChatBot()
chatbot.start_chat()

Chat here!
 what sound does a duck make?


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

CONTEXT:

Human: Hello
AI: What's up

Question: what sound does a duck make?[/INST]  A duck makes the sound "quack."
 what about penguins?


/opt/conda/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
